In [1]:
import pandas as pd
import numpy as np

import openai
from gensim.summarization import summarize

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import os

In [42]:
df = pd.read_csv('../data/raw/youtube/videos_2.csv')

In [43]:
df.head()

,title,description,tags,cleaned_transcript,raw_transcript,comments,comment_ids
0,The Millionaire Investing Advice For Teenagers,For anyone who wanted a video about how to inv...,"['how to invest', 'how to invest in stocks', '...",what's up you guys it's Graham here so I. have...,"[{'text': ""what's up you guys it's Graham here...",['Investing is a way to set aside money while ...,"['Ugwu-PbdD_35VymQsPh4AaABAg', 'Ugwu-PbdD_35Vy..."


In [44]:
def topTags(tags, n = 10):
    """Apply TF-IDF to the tags to get the ten most relevant tags"""
    corpus = " ".join([tag.strip().strip("\'") for tag in tags[1:-1].strip('[').strip(']').split(',')])
    
    # Should I just make one of these?
    # Convert the text into a sparse matrix using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([corpus])

    # Get the feature names and scores
    feature_names = vectorizer.get_feature_names_out()
    scores = dict(zip(feature_names, tfidf.data))

    # Sort the scores in descending order
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Select only the top N features
    selected_features = [x[0] for x in sorted_scores[:n]]
    
    return " ".join(selected_features)

In [45]:
def summarizeTranscript(transcript):
    """Cleans and summarizes transcript"""
    text = transcript.replace("\n", " ").replace(" - ", "").replace('- ', "").replace("\'", "").replace(".", ". ")
    if len(text) < 100:
        return text
    elif len(text.split(".")) <= 1:
        return text
    return summarize(text, word_count = 100)

In [46]:
def createPrompt(title, transcript, tags):
    """Creates the prompt"""
    return title + ' ' + transcript.replace("\n", " ") + ' ' + tags

In [49]:
print(createPrompt(df['title'][0], summarizeTranscript(df['cleaned_transcript'][0]), topTags(df['tags'][0])))

The Millionaire Investing Advice For Teenagers turn 18 go and get a credit card I want. get a credit card and thats gonna be. account without any ridiculous fees like. youre not making a lot of money right. youre not paying a lot in taxes thats. losing a lot of money to taxes like you. makes you more money its kind of like a. more money Im talking about like high. interest rate credit card debt a high. your money now when youre young is the. time to do it like if you want to go and. youre 40 years old so any learning. you your when build estate market what 20s credit invest


In [2]:
# Initialize the API client
openai.api_key = os.environ.get("OPEN_AI_KEY")
# Provide the model with instructions
model_engine = "text-davinci-002"
prompt = "Classify the following text as positive or negative: "

# Initial request to get the context
completions = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=0.5,
)

# Get the context from the initial request
context = completions.choices[0].text

# Function to make classification requests
def classify_text(text):
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
        context=context + text
    )
    return completions.choices[0].text

# Call the function to classify text
text = "This is a positive text."
classification = classify_text(text)
print(classification)

text = "This is a negative text."
classification = classify_text(text)
print(classification)


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.